# Gathering Stats about Latvian Periodicals

We will explore zip files that store plaintext files of Latvian periodicals. We will gather some statistics about the periodicals and their content.

## Years published

Our first task will be to extract years from the filenames of the periodicals. We will use regular expressions to extract the years from the filenames.

The file names are stored in zip files. We will use the `zipfile` module to extract the filenames from the zip files.

In [51]:
# first get Python version
import sys
print(f"Python version: {sys.version}")
from pathlib import Path
from datetime import datetime
print(f"Current date and time: {datetime.now()}")
# computer CPU type
import platform
print(f"Computer processor: {platform.processor()}")
# print current working directory
print(f"Current working directory: {Path.cwd()}")
import zipfile
import re
import pandas as pd
print(f"Pandas version: {pd.__version__}")
from tqdm import tqdm
# let's import visualization library Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly import __version__ as plotly_version
print(f"Plotly version: {plotly_version}")


Python version: 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
Current date and time: 2024-06-11 12:54:33.557058
Computer processor: Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
Current working directory: c:\Users\vsaules\Github\lnb_transports\notebooks
Pandas version: 2.2.1
Plotly version: 5.19.0


In [8]:
src_folder = Path("I:/zips")
assert src_folder.exists(), f"Source folder {src_folder} does not exist"
print(f"Source folder: {src_folder}")
# get list of files that contain word articles in them
files = list(src_folder.glob("*articles*.zip"))
print(f"Number of files: {len(files)}")


Source folder: I:\zips
Number of files: 118


In [33]:
# now we will write a function that will extract file names from a single zip file
def get_file_names(zip_file: Path, skip=1) -> list:
    with zipfile.ZipFile(zip_file, "r") as z:
        # get rid of folder names - we know first one is generally folder
        file_paths = z.namelist()[skip:]
        # now we only want file paths without folders
        file_names = [Path(file_path).name for file_path in file_paths]
        return file_names
z_file = files[0]
print(f"File: {z_file}")
file_names = get_file_names(z_file)
print(f"Number of files in zip: {len(file_names)}")

File: I:\zips\adelaides_latviesu_zinotajs_articles.zip
Number of files in zip: 725


In [22]:
# first 3 filenames
print(file_names[:3])

['xalz1962n114_001_plaintext_s01.txt', 'xalz1962n114_001_plaintext_s02.txt', 'xalz1962n114_002_plaintext_s03.txt']


In [26]:
# now let's write a function that will take a single file name and extract following two pieces of information:
# title - which is any text before year
# year - which is first 4 digit number in the file name
# we will use regular expression to extract year from the file name
def get_title_and_year(file_name: str) -> tuple:
    # regular expression to extract year
    year = re.search(r"\d{4}", file_name).group()
    # if length of year is not 4, return None
    if len(year) != 4:
        return {}
    # title is any text before year
    title = file_name.split(year)[0].strip()
    return {"title": title, "year": int(year)}
# let's test this function on first file name
d = get_title_and_year(file_names[0])
print(f"Title: {d['title']}, Year: {d['year']}")


Title: xalz, Year: 1962


In [28]:
# now let us write a function that given filenames will return a list of dictionaries
# each dictionary will have two keys: title and year
def get_titles_and_years(file_names: list) -> list:
    return [get_title_and_year(file_name) for file_name in file_names]
# let's test this function
titles_and_years = get_titles_and_years(file_names)
# first 3 titles and years
print(titles_and_years[:3])
# last 3 titles and years
print(titles_and_years[-3:])

[{'title': 'xalz', 'year': 1962}, {'title': 'xalz', 'year': 1962}, {'title': 'xalz', 'year': 1962}]
[{'title': 'xalz', 'year': 2003}, {'title': 'xalz', 'year': 2003}, {'title': 'xalz', 'year': 2003}]


In [48]:
# now let us write a function that will take list of dictionaries of  years and titles
# additional arguments will be start_year with default value of 1920 and end_year with default value of 1940
# this function will convert this list into pandas DataFrame and extract following information:
# min_year, max_year, total count of publications, count of publications between start_year and end_year (inclusive)
def get_summary(titles_and_years: list, start_year=1920, end_year=1940) -> dict:
    df = pd.DataFrame(titles_and_years)
    min_year = df["year"].min()
    max_year = df["year"].max()
    total_count = df.shape[0]
    count_between = df.query(f"{start_year} <= year <= {end_year}").shape[0]
    return {"min_year": min_year, "max_year": max_year, "total_count": total_count, f"count_{start_year}_{end_year}": count_between}
# let's test this function
summary = get_summary(titles_and_years)
print(summary)

{'min_year': 1962, 'max_year': 2003, 'total_count': 725, 'count_1920_1940': 0}


In [49]:
# let's create a function that given a zip file will return summary of all files in that zip
# in addition the dictionary will contain key publication that will contain zip file stem
def get_zip_summary(zip_file: Path, 
                    remove="_articles",
                    start_year = 1920,
                    end_year = 1940) -> dict:
    file_names = get_file_names(zip_file)
    titles_and_years = get_titles_and_years(file_names)
    summary = get_summary(titles_and_years, start_year=start_year, end_year=end_year)
    summary["publication"] = zip_file.stem
    # let's remove from publication name
    summary["publication"] = summary["publication"].replace(remove, "")
    # let us also add full zip file path without PosixPath using forward slashes
    summary["zip_file"] = zip_file.as_posix()
    return summary
# let's test this function
zip_summary = get_zip_summary(z_file)
print(zip_summary)

{'min_year': 1962, 'max_year': 2003, 'total_count': 725, 'count_1920_1940': 0, 'publication': 'adelaides_latviesu_zinotajs', 'zip_file': 'I:/zips/adelaides_latviesu_zinotajs_articles.zip'}


In [50]:
# now let us write a function that will take a list of zip files
# it will return a pandas DataFrame with summary of all files in all zip files
def get_all_zips_summary(zip_files: list,
                            start_year = 1920,
                            end_year = 1940,
                          csv_file = Path("../csv/articles_publications_summary.csv"),
                          excel_file = Path("../xlsx/articles_publications_summary.xlsx"),
                          parquet_file = Path("../parquet/articles_publications_summary.parquet")) -> pd.DataFrame:
    summaries = [get_zip_summary(zip_file, start_year=start_year, end_year=end_year) for zip_file in tqdm(zip_files)]
    df = pd.DataFrame(summaries)
    # let's  have first column be publication
    df = df[["publication"] + [col for col in df.columns if col != "publication"]]
    if csv_file:
        # create all folders in the path if they do not exist
        csv_file.parent.mkdir(parents=True, exist_ok=True)
        print(f"Saving CSV file to {csv_file}")
        df.to_csv(csv_file, index=False)
    if excel_file:
        # create all folders in the path if they do not exist
        excel_file.parent.mkdir(parents=True, exist_ok=True)
        print(f"Saving Excel file to {excel_file}")
        df.to_excel(excel_file, index=False)
    if parquet_file:
        # create all folders in the path if they do not exist
        parquet_file.parent.mkdir(parents=True, exist_ok=True)
        print(f"Saving Parquet file to {parquet_file}")
        df.to_parquet(parquet_file, index=False)
    return df
# let's test this function
df = get_all_zips_summary(files)
# shape
print(df.shape)
# first 5 rows
df.head()

100%|██████████| 118/118 [02:06<00:00,  1.07s/it]


Saving CSV file to ..\csv\articles_publications_summary.csv
Saving Excel file to ..\xlsx\articles_publications_summary.xlsx
Saving Parquet file to ..\parquet\articles_publications_summary.parquet
(118, 6)


,publication,min_year,max_year,total_count,count_1920_1940,zip_file
0,adelaides_latviesu_zinotajs,1962,2003,725,0,I:/zips/adelaides_latviesu_zinotajs_articles.zip
1,australijas_latvietis,1949,2011,93179,0,I:/zips/australijas_latvietis_articles.zip
2,avangards_daugavpils,1962,1992,77967,0,I:/zips/avangards_daugavpils_articles.zip
3,avots,1905,1915,7567,0,I:/zips/avots_articles.zip
4,avots_latvijas_rakstnieku_savienibas_zurnals,1987,1992,1188,0,I:/zips/avots_latvijas_rakstnieku_savienibas_z...


In [44]:
# let us save summary to a CSV file
csv_folder = Path("../csv")
# create folder if it does not exist
csv_folder.mkdir(exist_ok=True)
csv_file = csv_folder / "articles_publications_summary.csv"
# let's  have first column be publication
df = df[["publication"] + [col for col in df.columns if col != "publication"]]
df.to_csv(csv_file, index=False)

In [45]:
# let's also save to excel
excel_folder = Path("../excel")
# create folder if it does not exist
excel_folder.mkdir(exist_ok=True)
excel_file = excel_folder / "articles_publications_summary.xlsx"
df.to_excel(excel_file, index=False)